In [ ]:
#Этот код принимает на вход текст в формате txt и выдает следующие метрики: оценка удобочитаемости нейросетевой моделью, оценка удобочитаемости по классическому автоматизированному индексу (на основе средней длины слов и предложений), 44 лингвистические характеристики текста, значимые для принятия решения об удобочитаемости. 

#Среда Python 3.7.9 64-bit

In [1]:
#необходимые библиотеки
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU, SpatialDropout1D, Concatenate
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
import pandas as pd
import keras
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
import json
import io 
from tensorflow.python.keras.preprocessing.text import tokenizer_from_json
import joblib
ss=joblib.load('std_scaler.bin')
import textstat
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import os.path
import sys
import shelve
import itertools
import spacy
import stanza
stanza.download('ru')
nlp2 = stanza.Pipeline('ru') 
from spacy.lang.ru.examples import sentences 
nlp = spacy.load('ru_core_news_lg') #словарь 
import re
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import string
from string import digits
import ruts
from ruts import BasicStats
from ruts import DiversityStats
import statistics 
from sklearn.model_selection import train_test_split
import string


D:\Python\379\sklearn\base.py:338: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Семейный\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2022-11-24 16:39:37 INFO: Downloading default packages for language: ru (Russian)...
2022-11-24 16:39:39 INFO: File exists: C:\Users\Семейный\stanza_resources\ru\default.zip.
2022-11-24 16:39:44 INFO: Finished downloading models and saved to C:\Users\Семейный\stanza_resources.
2022-11-24 16:39:44 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2022-11-24 16:39:44 INFO: Use device: cpu
2022-11-24 16:39:44 INFO: Loading: tokenize
2022-11-24 16:39:45 INFO: Loading: pos
2022-11-24 16:39:45 INFO: Loading: lemma
2022-11-24 16:39:45 INFO: Loading: depparse
2022-11-24 16:39:46 INFO: Loading: ner
2022-11-24 16:39:48 INFO: Done loading processors!


In [2]:
#Необходимые словари

A1 = ('а','август','автобус','автор','администрация','адрес','активный','алло','английский','англичанин','англичане','англичанка','англо-русский','испанско-русский','китайско-русский','немецко-русский','французско-русский','анкета','апельсин','аппетит','апрель','аптека','артист','артистка','архитектор','архитектура','аспирант','аспирантка','апсирантура','аудитория','аэропорт','бабушка','багаж','балет','банк','баскетбол','бассейн','бедный','бежать','без','бейсбол','белый','берег','бесплатный','библиотека','бизнес','бизнесмен','билет','биография','биолог','биология','бить','благодарить','близко','бог','богатый','болен','больна','больны','больше','большой','брат','брать','будущий','буква','булочная','бумага','бутерброд','бутылка','бывать','быстро','быстрый','быть','в','во','вагон','важно','важный','варить','ваш','вверх','вдруг','верх','везде','везти','век','великий','верить','вернуться','весело','возвращаться','весенний','веселый','весна','весной','весь','вся','все','ветер','вечер','вечером','вещь','взять','видео','видеть','виза','вилка','вино','висеть','включать','вкусно','вкусный','включить','вместе','вниз','внимание','внимательно','внимательный','внук','вкучка','время','вовремя','вода','водить','водка','возвращаться', 'воздух','возить','возможно','войти','входить','вокзал','волейбол','волосы','вопрос','восемнадцать','восемь','восемьдесят','восемьсот','воскресенье','восток','восточный','восьмой','вот','вперед','враг','врач','время','всегда','всего','вспоминать','вспомнить','вставать','встать','встретить','встречать','встретиться','встречаться','встреча','встречать','встретить','вторник','второй','вход','входить','войти','вчера','вчерашний','вы','выбирать','выбрать','выйти','выходить','выключать','выключить','вымыть','вырасти','высокий','высоко','выставка','выступать','выступить','выучить','выход','выходить','выйти','газета','галерея','где','географ','геограический','география','герой','гитара','главный','глаз','глубокий','говорить','сказать','год','голова','голос','голубой','гора','город','городской','горький','горячий','господин','госпожа','господа','гостиница','гость','государственный','государство','готов','готовить','приготовить','градус','граница','грамм','грипп','громко','группа','грустно','грустный','грязный','гулять','да','давай','давайте','дать','давать','давно','даже','далеко','далекий','дарить','подарить','дата','два','две','двадцать','двенадцать','дверь','двести','двое','дворец','девочка','девушка','девяносто','девятнадцать','девятый','девять','девятьсот','дедушка','декабрь','делать','сделать','дело','день','деньги','депутат','деревня','дерево','держать','десятый','десять','дети','детский','детство','дешевый','дешево','диалог','диван','директор','длинный','днем','до','добрый','доволен','доехать','дождь','доказать','доказывать','документ','долго','должен','дом','дома','домашний','домой','домохозяйка','дорога','дорого','дорогой','свидание','дочь','друг','другой','дружба','дружный','думать','подумать','душ','дядя','европейский','его','ее','еда','ездить','если','есть','ехать','еще','елка','жалко','жаль','жарко','ждать','подождать','желать','пожелать','жена','женат','жениться','женский','женщина','желтый','жив','живой','живопись','животное','жизнь','житель','жить','прожить','журнал','журналист','за','забывать','забыть','завтра','завтрак','завтракать','позавтракать','завод','задавать','задать','задание','задача','заказать','заказывать','заканчивать','закончить','закон','закрывать','закрыть','закрыт','зал','замечательный','замечать','заметить','замуж','замужем','заниматься','заняться','занят','занятие','запад','западный','заплатить','платить','запоминать','запомнить','зачем','звать','звонить','позвонить','звонок','звук','здание','здесь','здоров','здоровый','здравствуйте','здравствуй','зеленый','земля','зеркало','зима','зимний','зимой','злой','знакомиться','знакомить','познакомить','познакомиться','знаком','знакомый','знать','узнавать','значение','значить','золото','золотой','зонт','зонтик','зоопарк','зуб','и','игра','играть','сыграть','выиграть','переиграть','игрушка','идти','из','известный','извините','извини','изменить','изменять','изучать','изучить','или','имя','инженер','иногда','иностранец','иностранка','иностранный','институт','интересно','интересный','интересоваться','интересовать','интернет','искать','искусство','испанец','испанцы','испанка','испанский','историк','исторический','история','их','июль','июнь','к','ко','кабинет','каждый','казаться','как','какой','каникулы','карандаш','карта','картина','картофель','касса','кафе','каша','квартира','килограмм','километр','кино','киоск','китаец','китайцы','китайский','китаянка','класс','класть','положить','клуб','ключ','книга','книжный','когда','колбаса','команда','комедия','комната','композитор','компьютер','конверт','конец','конечно','конфета','концерт','кончать','кончить','кончаться','кончиться','копейка','корабль','коридор','коричневый','короткий','космический','космонавт','космос','костюм','который','кофе','кошка','красивый','красный','кресло','кровать','круглый','ксерокс','кто','кто-то','куда','культура','купить','покупать','курить','курица','куры','курс','куртка','кухня','лампа','левый','легко','лежать','лекарство','лекция','лес','лестница','летать','лететь','летний','лето','летом','легкий','лимон','литература','лифт','лицо','лоб','ложка','луна','лучше','хорошо','хороший','лыжи','лыжа','любимый','любить','любовь','люди','магазин','магнитофон','май','маленький','мало','мальчик','мама','марка','март','масло','математик','математика','мать','машина','медицина','медленно','медленный','медсестра','международный','менеджер','меньше','менять','поменять','место','месяц','метр','метро','мечта','мешать','мечтать','помешать','милиционер','милиция','миллион','минус','минутка','минута','минуточка','мир','мирный','младший','многие','много','больше','можный','может',
'можно','мой','моя','мои','молодежный','молодежь','молодец','молодцы','молодой','молодость','молоко','молчать','момент','море','мороженое','москвич','москвичка','московский','москва','мочь','смочь','муж','мужской','мужчина','музей','музыка','музыкальный','музыкант','мультфильм','мы','мыло','мыть','вымыть','мягкий','мясо','мяч','на','наверное','надеяться','надо','назад','называться','найти','находить','наконец','налево','написать','писать','направо','напротив','нарисовать','рисовать','народ','народный','наука','научиться','учиться','научный','находить','найти','находиться','национальный','начало','начать','начинать','начинаться','начаться','наш','не','небо','невозможно','недавно','недалеко','неделя','нельзя','немец','немцы','немецкий','немка','несколько','несчастный','несчастье','нет','только','нигде','никогда','никто','никуда','ничего','ничто','но','новость','новый','нога','нож','ноль','номер','нулевой','нормально','нос','носок','носки','ночь','ночью','ноябрь','нравиться','понравиться','нужен','нужны','нужный','о','об','обед','обедать','пообедать','обещать','пообещать','образование','обувь','общежитие','общество','общий','объявление','объяснять','объяснить','обыкновенный','обычно','обычный','обязательство','овощ','овощи','овощной','огромный','одеваться','одеться','надеть','одежда','один','одна','одно','одинаковый','одиннадцать','однажды','озеро','оканчивать','окончить','окно','около','октябрь','он','она','оно','они','опаздывать','опоздать','опасно','опера','оптимист','опытный','опять','организовать','организовывать','осенний','осень','осенью','лсматривать','осмотреть','особенно','остановка','осторожно','остров','ответ','ответить','отвечать','отдохнуть','отдыхать','отдых','отец','открывать','открыть','открываться','открыться','открыт','открытка','откуда','отлично','отсюда','оттуда','отчество','очень','очки','ошибка','палец','пальто','памятник','папа','парк','паспорт','певец','певица','пенсионер','пенсионерка','пенсия','первый','перевести','переводить','перевод','переводчик','передавать','передать','передача','радиопередача','телепередача','перейти','переходить','перерыв','переход','песня','петь','печенье','пешком','пианино','пиво','писатель','писать','написать','письмо','пить','выпить','плавать','план','платить','заплатить','платье','плащ','плеер','плохо','хуже','плохой','площадь','плыть','плюс','по','по-английски','победа','победить','поблагодарить','благодарить','побывать','бывать','поверить','верить','повторить','повторять','погода','подарить','дарить','подарок','подождать','подруга','подумать','поезд','поездка','поехать','пожалуйста','позвать','позвонить','звонить','поздно','поздравить','поздравлять','познакомиться','по-испански','пойти','пока','показать','показывать','по-китайски','покупать','купить','пол','поле','полезный','поликлиника','политика','политический','полка','половина','положить','класть','полтора','получать','получить','полюбить','помнить','помогать','помочь','по-моему','понедельник','по-немецки','понимать','понять','понравиться','нравиться','понятно','пообедать','попросить','популярный','по-русски','послать','посылать','после','последний','послушать','слушать','посмотреть','смотреть','посоветовать','советовать','посольство','поставить','ставить','построить','строить','поступать','поступить','посылать','послать','потерять','терять','потом','потому','поужинать','ужинать','по-французски','почему','почта','почти','поэзия','поэт','поэтому','появиться','появляться','прав','права','правы','правда','правильно','правительство','правый','праздник','предмет','президент','прекрасно','прекрасный','преподаватель','преподавательница','преподавать','привет','пригласить','приглашать','приезд','приезжать','приехать','прийти','приходить','пример','природа','причина','приятно','проблема','программа','продавать','продать','продавец','продавщица','продолжать','продолжить','продолжаться','продукты','пройти','просить','попросить','проспект','простой','профессия','профессор','процент','прочитать','читать','прошлый','прямо','психолог','психология','птица','путешествовать','пятнадцать','пятница','пятый','пять','пятьдесят','пятьсот','работа','работать','рабочий','рад','рады','рада','радио','радость','раз','разговаривать','размер','разный','район','рано','раньше','рассказ','рассказать','рассказывать','расти','вырасти','ребенок','дети','редко','режиссер','результат','река','религия','ресторан','решать','решить','рис','рисовать','нарисовать','родина','родители','родиться','родной','рождение','роль','роман','российский','рот','рубашка','рубль','рука','русский','русская','ручка','рыба','рынок','рядом','с','со','сад','садиться','сесть','сажать','салат','сам','самолет','самостоятельно','самый','сантиметр','сапоги','сахар','цвет','свет','светлый','свободный','свой','своя','свое','сдавать','сдать','сделать','себя','север','северный','сегодня','седьмой','сейчас','секретарь','секунда','семнадцать','семь','семьдесят','семьсот','семья','сентябрь','сердце','серый','серьезно','серьезный','сестра','сигарета','сидеть','сильный','симпатичный','синий','система','сказать','говорить','сказка','сколько','скоро','скрипка','скучно','сладкий','слева','следующий','словарь','слово','случай','случаться','случиться','слушать','слышать','смелый','смерть','смешной','смеяться','смотреть','посмотреть','смочь','мочь','сначала','снег','снова','собака','собирать','собрать','собор','собрание','совет','советовать','посоветовать','современный','согласен','согласный','сок','солнце','соль','сообщать','сообщить','сообщение','сорок','сосед','соседка','соседний','спасибо','спать','спеть','петь','специалист','специальность','спокойно','спокойный','спорт','спортивный','спортсмен','справа','спрашивать','спросить','сразу','среда','средний','стадион','стакан','станция','старость','старший','старый','стать','статья','стена','стихи','сто','стоить','стол','столица','столовая','стоять','страна','страница','строитель','строительный','строить','построить','студент','студентка','студенческий','стул','суббота','сувенир','сумка','суп','сутки','счастливый','счастье','съесть','есть','сыграть','сын','сыр','сюда','так','также','такой','такая','такси','талант','талантливый','там','танец','танцевать','тарелка','твердый','твой','театр','текст','телевизор','телеграмма','телефон','температура','теннис','теперь','тепло','тетрадь','техника','технический','темный','теплый','тетя','тихо','товарищ','тогда','то','тоже','только','торт','та','те','точка','точно','трамвай','транспорт','тратить','третий','три','тридцать','тринадцать','триста','троллейбус','трудно','трудный','туда','турист','тут','туфли','ты','тысяча','тяжелый','у','убивать','убить','уважать','уважаемый','уверен','увидеть','видеть','ударение','удивительный','удобный','удовольствие','уезжать','уехать','уже','ужинать','ужин','узкий','уйти','уходить','улица','улыбаться','улыбнуться','умереть','уметь','суметь','умный','универмаг','университет','университетский','упасть','падать','упражение','урок','успех','уставать','устать','утро','ухо','утром','уходить','участвовать','учебник','учебный','ученик','ученица','учитель','учительница','учить','выучить','фабрика','факс','факультет','фамилия','февраль','фермер','физик','физика','физический','филолог','филологический','филология','философ','философия','философский','фильм','фирма','флешка','фонтан','фотоаппарат','фотографировать','фотография','фраза','француз','французский','француженка','фрукты','футбол','футболист','характер','химик','химический','химия','хлеб','хобби','ходить','хозяин','хозяйка','хоккеист','хоккей','холодно','холодный','хороший','хорошо','хотеть','художник','хуже','царь','цветной','цветы','целовать','целый','цена','центр','центральный','цирк','цифра','чай','чайник','час','часто','часть','часы','чашка','чей','чья','чье','чьи','человек','чем','чемпион','через','честный','четверг','четвертый','четыре','четыреста','четырнадцать','черный','число','чистый','читальный','читатель','читать','прочитать','чтение','что','чтобы','чувствовать','шапка','шарф','шахматист','шахматы','шестой','шестнадцать','шесть','шестьдесят','шестьсот','широкий','шкаф','школа','школьник','школьница','шоколад','шуметь','шумный','шутить','пошутить','шутка','щи','экзамен','экономика','экономист','экономический','экскурсия','экскурсовод','энергичный','энергия','этаж','этот','это','эта','эти','юбка','юг','южный','юмор','юность','юноша','юридический','юрист','я','яблоко','являться','ягода','язык','яйцо','январь','яркий')
B1 = ('абсолютно','автобусный','автомат','анализ','азиатский','академия','альбом','американец','американка','америка','ангина','анкета','ансамбль','армия','анализировать','аналитический','атмосфера','атом','атомный','афиша','балерина','балет','балкон','банан','бандероль','батон','башня','бегать','береза','беречь','сберегать','уберегать','беседа','беседовать','беспокоиться','обеспокоенный','благодаря','бланк','бледный','ближе','вблизи','близкий','блюдо','бой','более','болезнь','болельщик','больница','больно','борода','бороться','борщ','борьба','ботинок','ботинки','бояться','опасаться','брить','бриться','бросать','бросить','брюки','будущее','буфет','бы','ваза','бывший','ванная','вверху','вдвоем','вежливый','повезти','велик','велосипед','верный','верхний','вести','вечерний','виноград','власть','внизу','внутри','вовремя','вопервых','вовторых','водить','возраст','война','сходить','воевать','военный','вокзал','вокруг','волк','','втретьих','вчетвертых','впятых','воспитание','воспитать','воспитывать','воспоминание','впервые','впереди','впечатление','вредно','вредный','вслух','выезжать','выехать','вызвать','вызывать','выздоравливать','выздороветь','выиграть','выигрывать','вылечить','лечить','выпить','пить','выполнить','выполнять','выражать','выразить','вырасти','расти','','выступление','выше','газ','галстук','гараж','гастроном','гденибудь','гдето','генерал','геолог','геология','геологический','гимназия','гимнастка','гимнаст','гимнастика','глава','глагол','глубже','глупо','глупый','говядина','голодный','голос','гордиться','гордый','горе','гореть','горло','гостеприимный','гражданин','гражданка','гражданство','грамматика','граница','громкий','громче','грубый','грудь','грязно','гуманитарный','дальше','дальний','дача','дважды','двое','двигаться','движение','двойка','двор','действие','действительно','декан','декларация','делегация','делиться','разделиться','демократический','демонстрация','деревянный','детектив','деятель','джинсы','диктант','дипломат','дирижер','диск','диссертация','длина','добавить','добавлять','добиться','добиваться','доброта','договариваться','договориться','доезжать','доехать','дойти','доходить','доклад','докктор','документальный','долгий','доска','достигать','достигнуть','достаточно','дочка','древний','дружеский','дружить','душа','дышать','единственный','единый','ежегодно','ежегодный','ежедневно','ежедневный','жареный','же','желание','железный','желудок','жених','жестокий','живот','заболеть','болеть','заботиться','зависеть','закрыться','закрываться','закурить','закусить','закуска','заменить','заменять','замолчать','молчать','менять','занять','занимать','записка','записывать','записать','заплакать','запретить','запрещать','зарплата','зарубежный','засмеяться','затем','заходить','захотеть','зачем','зачет','защитить','защищать','заявление','заяц','звать','звезда','зверь','землетрясение','знак','знаменитый','знание','зритель','зубной','идея','изза','изобразить','изображать','известно','изучение','иметь','импорт','истратить','тратить','будто','каков','календарь','каменный','камень','капуста','карточка','кассир','катастрофа','кататься','кафедра','качество','кашель','кефир','классический','климат','когданибудь','когдато','когдалибо','количество','коллега','колледж','коллекция','кольцо','команда','командировка','конкурс','консерватория','конспектировать','конспект','конституция','консультация','контрольный','конференция','коньки','конек','кормить','покормить','коробка','корова','корпус','кот','котлета','кошелек','красиво','кремль','крепкий','крепче','крестьянин','кризис','крикнуть','кричать','критиковать','кровь','кроме','кроссовки','кроссовок','крупный','крыша','ктонибудь','ктолибо','ктото','курить','куртка','кухня','лаборатория','легче','лечить','лечь','ложиться','лед','летчик','ли','лист','литературный','лишний','ловить','поймать','лодка','ложка','ломать','сломать','лошадь','лук','любой','мал','меньше','маршрут','маршрутка','мастер','матч','мебель','медведь','между','мелодия','меньшинство','мертвый','миллиард','млрд','тыс','руб','млн','милый','мимо','министр','минус','мировой','митинг','мнение','мобильный','модель','монета','мокрый','мороз','морской','моряк','мост','мужественный','мужество','мысль','мягкий','наблюдать','наверх','наверху','навещать','навестить','навсегда','навстречу','награда','над','надевать','надеть','надежда','надолго','название','наиболее','наизусть','намного','наоборот','напечатать','печатать','напомнить','напоминать','направление','население','насколько','насморк','настроение','наступать','наступить','наука','научный','научить','учить','невеста','негде','независимость','некогда','некоторый','необходимо','немедленно','необыкновенный','неожиданно','непрерывно','неприятность','нервный','несколько','несмотря','на','нести','неужели','нефть','нечего','ни','нигде','никакой','ничей','но','новогодний','носить','ночной','ну','обидеть','обижать','обидеться','обижаться','обладать','облако','область','обмен','обойти','обходить','обрадовать','радовать','образец','образование','обратить','обращать','обращаться','обратиться','обсудить','обсуждать','обувь','обучать','обучить','обучение','обходить','общежитие','общественный','общество','общий','объединять','объединить','объявить','объявлять','объявление','объяснение','объяснять','объяснить','обычай','овца','огонь','огромный','огурец','однако','оказаться','оказываться','океан','окончание','окончить','оканчивать','опасность','организация','операция','описывать','описать','оригинальный','оружие','освобождать','освободить','особенность','особый','оставаться','остаться','оставлять','оставить','останавливаться','остановить','остановка','от','отдавать','отдать','отметка','отнимать','отнять','относиться','отношение','отойти','отходить','отсталый','отпуск','отъезд','офицер','официальный','официант','оформлять','оформить','оценивать','оценить','оценка','ошибаться','ошибиться','падать','упасть','падеж','память','парламент','партия','пассажир','педагогический','перевозить','перевезти','переговоры','перед','переезжать','переехать','перерыв','пересадка','перчатки','песок','пессимист','пирог','пирожок','пирожное','пицца','плечо','побежать','бежать','побеждать','погибать','погибнуть','под','подготовиться','готовиться','подготовка','поднимать','поднять','подниматься','подписать','подписывать','подойти','подходить','поесть','пожелать','пожениться','позавчера','позвонить','звонить','позже','показывать','показать','покупатель','полезно','поликлиника','политехнический','полный','половина','полотенце','полтора','пользоваться','поменять','менять','помешать','мешать','помидор','помоему','пообещать','обещать','попрежнему','попробовать','пробовать','поражение','порт','портрет','посетить','посещать','посидеть','последний','послезавтра','пословица','посол','постараться','стараться','постоянно','потвоему','потолок','потратить','похвалить','хвалить','похожий','похож','почувствовать','чувствовать','пошутить','шутить','правило','практика','практический','предлагать','предложить','представитель','прежде','чем','премия','привезти','привозить','приводить','привести','привыкать','привыкнуть','пригласить','приглашать','приготовить','готовить','приказать','приказывать','прилетать','прилететь','принадлежность','принести','приносить','принимать','принять','прислать','присылать','пробовать','попробовать','проверить','проверять','провести','проводить','провожать','проводить','прогресс','прогулка','продавец','продавщица','провожать','проводить','продать','продавать','продолжать','продолжить','продолжается','продолжиться','проезжать','проехать','произведение','производить','произвести','производство','произносить','произнести','произношение','промышленность','промышленный','простить','прощать','просыпаться','проснуться','против','прохладно','прохладный','проходить','пройти','процесс','прощаться','попрощаться','прыгать','прыгнуть','психологический','пустой','пусть','путешественник','путь','пытаться','попытаться','пьеса','пятерка','равен','равный','радовать','обрадовать','радоваться','обрадоваться','разве','развиваться','развитие','разделить','делить','разделиться','делиться','разговор','разница','разрешать','разрешить','расписание','ребята','революция','регулярно','редкий','реже','резать','результат','ректор','республика','рецепт','решительный','рисунок','ровно','род','родственник','рождество','розовый','роза','рояль','ругать','руководитель','руководить','рыбак','ряд','салют','самовар','сберечь','беречь','сварить','варить','свежий','свидание','свинина','свитер','свобода','свободен','связь','седой','секунда','сельский','хозяйство','семейный','семестр','семинар','серебро','серебряный','середина','серый','сессия','сибирский','сила','символ','ситуация','скидка','скорость','скромный','слабый','славянский','следовательно','слишком','сложный','сломать','ломать','слон','служить','смысл','событие','совет','советский','сожаление','совсем','создавать','создать','солдат','сомневаться','сон','соревнование','состоять','соус','сохранить','сохранять','социальный','спасти','спасать','спектакль','специальный','спешить','спина','спички','спорить','способный','спутник','сравнивать','сравнить','среди','ставить','поставить','становиться','стать','стараться','постараться','старик','старость','старше','старый','стипендия','страдать','постирать','столько','сторона','странный','страшный','строго','стройка','стыдно','судьба','сухой','существовать','сфотографировать','сцена','счет','считать','посчитать','сшить','шить','съезд','таблетка','тайна','такой','образ','таможня','творчество','тело','тема','темно','темный','теоретический','теория','теплоход','территория','терять','потерять','тихий','тишина','товар','толстый','тонкий','торговля','торт','зрение','точный','трава','трагедия','традиция','истратить','потратить','требовать','потребовать','тренироваться','тренировка','трое','тройка','труд','трудолюбивый','туман','туманный','тюрьма','убегать','убежать','убедить','убеждать','убирать','убрать','уважаемый','уважать','увезти','увозить','увеличивать','увеличиваться','увлекаться','увлечься','увлечение','угол','уголь','удаваться','удасться','ударить','удобно','ужасно','ужасный','узнавать','узнать','уйти','уходить','укол','улетать','улететь','улучшать','улучшить','улучшаться','улучшиться','улыбка','уменьшать','уменьшиться','умываться','умывать','умыться','уникальный','урожай','условие','услышать','слышать','успевать','успеть','успешно','успешный','успокаиваться','успокоиться','уставать','усталый','устно','устный','усы','утренний','училище','факт','фашизм','фашист','фестиваль','фигурный','катание','флаг','фонтан','форма','фронт','футбольный','хвалить','хватать','хватить','хирург','хозяйство','домашний','дом','хор','хотеться','хотя','храм','художественный','художник','худой','царь','целоваться','поцеловаться','цель','цена','ценный','центр','центральный','церковь','чейнибудь','чейто','человечество','чемодан','чемпионат','четверо','четверка','член','чтение','чутьчуть','чтото','чтонибудь','чужой','чудо','чувство','шашлык','шестнадцать','шея','шить','сшить','шляпа','шоссе','шофер','шум','шуметь','шумный','щека','щетка','экология','экологический','экономить','экспедиция','эксперимент','экспорт','электричество','электростанция','электроника','этап','эпоха','энциклопедия','ядерный','ясно','ясный')
VN1 = ('анализ','донос','захват','насос','отказ','подбор','приток','размен','созыв','блеск','допуск','заход','настил','отклик','подвиг','прихлоп','размер','состав','ввод','досмотр','защита','настой','откорм','поджог','приход','размыв','спад','ввоз','дрожь','звон','настрой','отлет','подкова','прицеп','разнос','сплав','взгляд','езда','извив','насыпь','отмена','подкоп','причал','разрез','спор','вздох','забег','изгиб','набор','отпуск','подкуп','прищур','разруха','спрос','взлет','забой','излом','недосмотр','отрез','подмена','приют','разрыв','сруб','взмах','забор','измена','обвал','отрыв','поднос','пробег','разъезд','срыв','взнос','забота','изморозь','обгон','отряд','подпись','провал','раскат','ссора','взор','завал','износ','обет','отсвет','подрыв','провод','раскол','ссуда','взрыв','завеса','импорт','обжиг','отсек','подступ','проводы','раскоп','стон','визг','завет','исповедь','обзор','отстой','подсчет','прогиб','раскрой','страда','вклад','зависть','испуг','обида','отступ','подход','прогноз','распад','стрекот','вкус','завод','исток','обман','отсчет','подъезд','прогон','расплав','стужа','возврат','завоз','исход','обмен','оттиск','подъем','прогул','расплата','стык','возглас','загар','итог','обмер','отход','позор','проезд','расправа','суета','возраст','загиб','клев','обмолот','отчет','поиск','происки','рассвет','сход','восторг','загон','клевета','оборона','отъезд','показ','пролет','рассказ','счет','вражда','задира','клей','обрез','охват','поклон','пролог','расспрос','съезд','вскрик','заезд','клеймо','обрыв','охота','покой','пролом','расстрел','торговля','всплеск','зажим','конспект','обряд','охрана','покос','промах','раствор','тормоз','встреча','заказ','контраст','обстрел','перебор','покров','пропись','растрата','травля','всхлип','закал','ловля','обхват','перевал','полет','пропуск','расход','трепет','всходы','закат','лом','обход','перевес','полив','прорезь','расцвет','треск','въезд','залежь','мах','объезд','перевод','помета','прорубь','расчет','убор','выбор','залив','набег','обыск','перевоз','помеха','прорыв','розыск','угар','выброс','залог','набор','ограда','перевыборы','пороша','просвет','рокот','уговор','вывих','замах','навал','ожог','перевязь','порча','просмотр','ропот','угроза','вывод','замена','навес','окись','перегрев','посол','простуда','роспись','удав','вывоз','замер','нагрев','оклик','передел','поступь','просчет','россыпь','удар','выговор','замес','надел','оковы','переезд','потеха','протест','рост','удел','выдох','замысел','надзор','окоп','перекат','поток','проток','сбор','уклад','выезд','занавес','надпись','окрик','перекись','похвала','протока','сброс','уклон','вызов','запас','надрез','опись','перелет','почет','проход','свист','укол','выкрик','запад','надрыв','опора','перелом','пощада','пряжа','сгиб','укор','выкуп','запал','нажива','оправа','перемена','преграда','разбег','сговор','укус','вылет','запас','нажим','опрос','перенос','прибой','разбор','сдвиг','улика','вымысел','запах','наказ','ориентир','перепад','привал','разброд','склад','умысел','выпас','запись','накал','осада','переплет','привет','развал','скорбь','уплата','выплата','запор','накат','оскал','переправа','привод','развод','скос','упор','выпуск','запрет','накипь','осмотр','перерасход','приговор','разворот','скрежет','упрек','вырез','запрос','наклон','отблеск','перерыв','приезд','разгар','скрип','уровень','высев','запруда','налет','отбой','пересев','прием','разговор','скука','урон','выслуга','запуск','налив','отбор','пересказ','призыв','разгон','слет','услуга','выстрел','зарок','налог','отброс','пересмотр','приказ','разгром','слом','уступ','выступ','заряд','намек','отвал','пересчет','приклад','разгул','смена','утрата','выхлоп','засада','намыв','ответ','перехват','прилет','раздел','смесь','уход','выход','засев','нанос','отвод','переход','примесь','раздор','смотр','учет','вычет','заслон','наплыв','отдел','побег','примета','разлад','смута','хвала','довод','заслуга','нарост','отдых','победа','прирост','разлив','смыв','хохот','договор','застава','нарыв','отек','повод','присмотр','разлом','смысл','хруст','доклад','засуха','наряд','отзвук','повтор','приступ','разлука','снаряд','шелест','домысел','затрата','наскок','отзыв','погоня','присяга','размах','снос','шум')
stopwords = ["c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]

Tikhonov = pd.read_csv("D:\\Program files\\R\\Temp\\CB\\DICT\\Tikhonov.csv", encoding = 'cp1251', header = 0, sep = '|', engine= 'python')
Tikhonov.columns = ['word','morph','num']
Tikhonov_morph = Tikhonov['morph']
Tikhonov_morph = Tikhonov_morph.tolist()
Tikhonov_morph = map(str, Tikhonov_morph)
S = []
for item in Tikhonov_morph:
    S.append(item.count('!'))
Tikhonov['numbers'] = S
Tikhonov = Tikhonov.drop('num', 1)
Tikhonov = Tikhonov.replace(r'\s+','',regex=True)
Tikh_dict = dict(zip(Tikhonov['word'] ,Tikhonov['numbers']))

#print(Tikh_dict['ящур'])

#загружаем словарь частотности и запоминаем 30% наименее частотных слов

dictionary = pd.read_csv("D:\\Program files\\R\\Temp\\CB\\DICT\\freqrnc2011.csv", encoding = 'utf-8', header = 0, sep='\\t', engine='python')

dictionary1 = dictionary.drop('PoS', 1)
dictionary1 = dictionary1.drop('R', 1)
dictionary1 = dictionary1.drop('D', 1)
dictionary1 = dictionary1.drop('Doc', 1)
dictionary1 = dictionary1.sort_values(by=['Freq(ipm)'])
dictionary2 = dictionary1[:15641] #здесь выбрано 30% наименее частотных слов
dictionary3 = dictionary2['Lemma']

full_dict = dictionary1['Lemma']
full_dict = full_dict .tolist()


dictionary_freq = dictionary3.tolist()

#Словари для эвфонического индекса 
neblag_v_slove_sogl = ("бв","бг","бд","бж","бз","бк","бл","бм","бн","бп","бр","бс","бт","бф","бх","бц","бч","бш","бщ","вб","вг","вд","вж","вз","вк","вл","вм","вн","вп","вр","вс","вт","вф","вх","вц","вч","вш","вщ","гб","гв","гд","гж","гз","гк","гл","гм","гн","гп","гр","гс","гт","гф","гх","гц","гч","гш","гщ","дб","дв","дг","дж","дз","дк","дл","дм","дн","дп","др","дс","дт","дф","дх","дц","дч","дш","дщ","жб","жв","жг","жд","жз","жк","жл","жм","жн","жп","жр","жс","жт","жф","жх","жц","жч","жш","жщ","зб","зв","зг","зд","зж","зк","зл","зм","зн","зп","зр","зс","зт","зф","зх","зц","зч","зш","зщ","кб","кв","кг","кд","кж","кз","кл","км","кн","кп","кр","кс","кт","кф","кх","кц","кч","кш","кщ","лб","лв","лг","лд","лж","лз","лк","лм","лн","лп","лр","лс","лт","лф","лх","лц","лч","лш","лщ","мб","мв","мг","мд","мж","мз","мк","мл","мн","мп","мр","мс","мт","мф","мх","мц","мч","мш","мщ","нб","нв","нг","нд","нж","нз","нк","нл","нм","нп","нр","нс","нт","нф","нх","нц","нч","нш","нщ","пб","пв","пг","пд","пж","пз","пк","пл","пм","пн","пр","пс","пт","пф","пх","пц","пч","пш","пщ","рб","рв","рг","рд","рж","рз","рк","рл","рм","рн","рп","рс","рт","рф","рх","рц","рч","рш","рщ","сб","св","сг","сд","сж","сз","ск","сл","см","сн","сп","ср","ст","сф","сх","сц","сч","сш","сщ","тб","тв","тг","тд","тж","тз","тк","тл","тм","тн","тп","тр","тс","тф","тх","тц","тч","тш","тщ","фб","фв","фг","фд","фж","фз","фк","фл","фм","фн","фп","фр","фс","фт","фх","фц","фч","фш","фщ","хб","хв","хг","хд","хж","хз","хк","хл","хм","хн","хп","хр","хс","хт","хф","хц","хч","хш","хщ","цб","цв","цг","цд","цж","цз","цк","цл","цм","цн","цп","цр","цс","цт","цф","цх","цч","цш","цщ","чб","чв","чг","чд","чж","чз","чк","чл","чм","чн","чп","чр","чс","чт","чф","чх","чц","чш","чщ","шб","шв","шг","шд","шж","шз","шк","шл","шм","шн","шп","шр","шс","шт","шф","шх","шц","шч","шщ","щб","щв","щг","щд","щж","щз","щк","щл","щм","щн","щп","щр","щс","щт","щф","щх","щц","щч","щш")
neblag_v_slove_glasn = ("аа","ае","аё","аи","ао","ау","аэ","аю","ая","аы","еа","ее","её","еи","ео","еу","еэ","ею","ея","еы","ёа","ёе","ёё","ёи","ёо","ёу","ёэ","ёю","ёя","ёы","иа","ие","иё","ии","ио","иу","иэ","ию","ия","иы","оа","ое","оё","ои","оо","оу","оэ","ою","оя","оы","уа","уе","уё","уи","уо","уу","уэ","ую","уя","уы","эа","эе","эё","эи","эо","эу","ээ","эю","эя","эы","юа","юе","юё","юи","юо","юу","юэ","юю","юя","юы","яа","яе","яё","яи","яо","яу","яэ","яю","яя","яы","ыа","ые","ыё","ыи","ыо","ыу","ыэ","ыю","ыя","ыы")
schip = ('ж','ш','ч','щ')
NDU_dict = neblag_v_slove_sogl + neblag_v_slove_glasn + schip
new_list_b1 = []
for item in neblag_v_slove_sogl:
    new = 'б ' + item
    new_list_b1.append(new)
new_list_b2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' б'
    new_list_b2.append(new)
new_list_v1 = []
for item in neblag_v_slove_sogl:
    new = 'в ' + item
    new_list_v1.append(new)
new_list_v2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' в'
    new_list_v2.append(new)
new_list_g1 = []
for item in neblag_v_slove_sogl:
    new = 'г ' + item
    new_list_g1.append(new)
new_list_g2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' г'
    new_list_g2.append(new)
new_list_d1 = []
for item in neblag_v_slove_sogl:
    new = 'д ' + item
    new_list_d1.append(new)
new_list_d2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' д'
    new_list_d2.append(new)
new_list_zh1 = []
for item in neblag_v_slove_sogl:
    new = 'ж ' + item
    new_list_zh1.append(new)
new_list_zh2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' ж'
    new_list_zh2.append(new)
new_list_z1 = []
for item in neblag_v_slove_sogl:
    new = 'з ' + item
    new_list_z1.append(new)
new_list_z2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' з'
    new_list_z2.append(new)
new_list_k1 = []
for item in neblag_v_slove_sogl:
    new = 'к ' + item
    new_list_k1.append(new)
new_list_k2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' к'
    new_list_k2.append(new)
new_list_l1 = []
for item in neblag_v_slove_sogl:
    new = 'л ' + item
    new_list_l1.append(new)
new_list_l2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' л'
    new_list_l2.append(new)
new_list_m1 = []
for item in neblag_v_slove_sogl:
    new = 'м ' + item
    new_list_m1.append(new)
new_list_m2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' м'
    new_list_m2.append(new)
new_list_n1 = []
for item in neblag_v_slove_sogl:
    new = 'н ' + item
    new_list_n1.append(new)
new_list_n2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' н'
    new_list_n2.append(new)
new_list_p1 = []
for item in neblag_v_slove_sogl:
    new = 'п ' + item
    new_list_p1.append(new)
new_list_p2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' п'
    new_list_p2.append(new)
new_list_r1 = []
for item in neblag_v_slove_sogl:
    new = 'р ' + item
    new_list_r1.append(new)
new_list_r2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' р'
    new_list_r2.append(new)
new_list_s1 = []
for item in neblag_v_slove_sogl:
    new = 'с ' + item
    new_list_s1.append(new)
new_list_s2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' с'
    new_list_s2.append(new)
new_list_t1 = []
for item in neblag_v_slove_sogl:
    new = 'т ' + item
    new_list_t1.append(new)
new_list_t2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' т'
    new_list_t2.append(new)
new_list_f1 = []
for item in neblag_v_slove_sogl:
    new = 'ф ' + item
    new_list_f1.append(new)
new_list_f2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' ф'
    new_list_f2.append(new)
new_list_h1 = []
for item in neblag_v_slove_sogl:
    new = 'х ' + item
    new_list_h1.append(new)
new_list_h2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' х'
    new_list_h2.append(new)
new_list_c1 = []
for item in neblag_v_slove_sogl:
    new = 'ц ' + item
    new_list_c1.append(new)
new_list_c2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' ц'
    new_list_c2.append(new)
new_list_ch1 = []
for item in neblag_v_slove_sogl:
    new = 'ч ' + item
    new_list_ch1.append(new)
new_list_ch2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' ч'
    new_list_ch2.append(new)
new_list_sh1 = []
for item in neblag_v_slove_sogl:
    new = 'ш ' + item
    new_list_sh1.append(new)
new_list_sh2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' ш'
    new_list_sh2.append(new)
new_list_sch1 = []
for item in neblag_v_slove_sogl:
    new = 'щ ' + item
    new_list_sch1.append(new)
new_list_sch2 = []
for item in neblag_v_slove_sogl:
    new =  item + ' щ'
    new_list_sch2.append(new)
NDT1 = new_list_b1 + new_list_b2 + new_list_v1 + new_list_v2 + new_list_g1 + new_list_g2 + new_list_d1 + new_list_d2 + new_list_zh1 + new_list_zh2 + new_list_z1 + new_list_z2 + new_list_k1 + new_list_k2 + new_list_l1 + new_list_l2 + new_list_m1 + new_list_m2 + new_list_n1 + new_list_n2 + new_list_p1 + new_list_p2 + new_list_r1 + new_list_r2 + new_list_s1 + new_list_s2 + new_list_t1 + new_list_t2 + new_list_f1 + new_list_f2 + new_list_h1 + new_list_h2 + new_list_c1 + new_list_c2 + new_list_ch1 + new_list_ch2 + new_list_sh1 + new_list_sh2 + new_list_sch1 + new_list_sch2 
NDT2 = ["а а","а е","а ё","а и","а о","а у","а э","а ю","а я","а ы","е а","е е","е ё","е и","е о","е у","е э","е ю","е я","е ы","ё а","ё е","ё ё","ё и","ё о","ё у","ё э","ё ю","ё я","ё ы","и а","и е","и ё","и и","и о","и у","и э","и ю","и я","и ы","о а","о е","о ё","о и","о о","о у","о э","о ю","о я","о ы","у а","у е","у ё","у и","у о","у у","у э","у ю","у я","у ы","э а","э е","э ё","э и","э о","э у","э э","э ю","э я","э ы","ю а","ю е","ю ё","ю и","ю о","ю у","ю э","ю ю","ю я","ю ы","я а","я е","я ё","я и","я о","я у","я э","я ю","я я","я ы","ы а","ы е","ы ё","ы и","ы о","ы у","ы э","ы ю","ы я","ы ы"]
NDT_dict = NDT1 + NDT2


D:\Python\379\ipykernel_launcher.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
D:\Python\379\ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
D:\Python\379\ipykernel_launcher.py:29: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
D:\Python\379\ipykernel_launcher.py:30: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
D:\Python\379\ipykernel_launcher.py:31: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [3]:
#Функции для извлечения лингвистических характеристик

#1 Средняя длина предложения в словах
def SenLen(text):
    words = textstat.lexicon_count(text, removepunct=True)
    number_of_sentences = sent_tokenize(text)
    sentences = len(number_of_sentences)
    return float(words / sentences)

#2 Средняя длина дерева  синтаксического разбора на предложение
def tree_height(root):
    if not list(root.children):
        return 1
    else:
        return 1 + max(tree_height(x) for x in root.children)
def PTH(text):
    if type(text) == str:
        doc = nlp(text)
    else:
        doc = text
    roots = [sent.root for sent in doc.sents]
    return np.mean([tree_height(root) for root in roots])

#3 Среднее число подчинительных союзов на предложение
def SubCon(text):
   patternSC = r'\b(,\sчто\s|\sкак\s|чтобы\s|когда\s|\sпока\s|едва\s|с\sтех\sпор\sкак\s|лишь|\sпотому\sчто\s|потому,\sчто|\sтак\sкак\s|ибо\s|ввиду\s|\sесли\s|\sколи\s|\sкабы\s|\sхотя\s|\sнесмотря\sна\s|\sвопреки\s|\sпоэтому\s|\sчем\s|словно\s|подобно\s)\b'
   text_string = text.lower()
   number_of_sentences = sent_tokenize(text)
   match_pattern = re.findall(patternSC, text_string)
   return float (len(match_pattern) / len(number_of_sentences))

#4 Среднее число сочинительных союзов на предложение
def CoordC(text):
   patternCC = r'\b(,\sи\s|,\sа\s|,\sно\s|не\sтолько|так\sи|зато\s|тоже|также|однако|или)\b'
   text_string = text.lower()
   number_of_sentences = sent_tokenize(text)
   match_pattern = re.findall(patternCC, text_string)
   return float (len(match_pattern) / len(number_of_sentences))

#5 Доля именных деревьев в среднем на предложение
def NP(text):
    number_of_sentences = sent_tokenize(text)
    text = nlp(text)
    S = []
    for sent in text.sents:
        if sent.root.pos_ == 'NOUN':
            S.append(sent.root)
    return float (len(S) / len(number_of_sentences))

#6 Доля глагольных деревьев в среднем на предложение
def VP(text):
    number_of_sentences = sent_tokenize(text)
    text = nlp(text)
    S = []
    for sent in text.sents:
        if sent.root.pos_ == 'VERB':
            S.append(sent.root)
    return float (len(S) / len(number_of_sentences))


#7 Среднее число цифр на предложение
def NUM(text):
    number_of_sentences = sent_tokenize(text)
    text = nlp(text)
    S = []
    for token in text:
        if token.pos_ == 'NUM':
            S.append(token)
    return float (len(S) / len(number_of_sentences))

#8 Среднее число предложений, отягощенных причастиями и деепричастиями
def DiffSynt(text):
    number_of_sentences = sent_tokenize(text)
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.tag.POS)
    S = []
    for i in res:
        if i == 'PRTF' or i == 'PRTS' or i == 'GRND':
            S.append(i)
    return float(len(S) / len(number_of_sentences))

#9 Доля предложений, длина которых превышает 25 слов
def LSDaoust(text):
    number_of_sentences = sent_tokenize(text)
    test = sent_tokenize(text)
    S = []
    for sent in test:
        words = textstat.lexicon_count(sent)
        S.append(words)
    A = []
    for number in S:
        if int(number) > 25:
            A.append(number)
    return float (len(A) / len(number_of_sentences) ) 

#10 Среднее число слов до подлежащего или сказуемого на предложение
def PPos(text):
    number_of_sentences = sent_tokenize(text)
    test = sent_tokenize(text)
    S = []
    for sentence in test:
        nlp_sentence = nlp(sentence)
        for sent in nlp_sentence.sents:
            S += [sent.root.i]
  
    return float (sum(S) / len(S))

#11 Доля «:», «(», «)» и «;» в знаках препинания
def PrepR(text):
    test = nlp(text)
    S = []
    for token in test:
        if token.pos_ == 'PUNCT':
            S.append(token)

    semicilons = text.count(';')
    colons = text.count(':')
    parentheses1 = text.count('(')
    parentheses2 = text.count(')')
    parentheses = (parentheses1 + parentheses2)/2

    total = semicilons + colons + parentheses

    return float (total / len(S))

#12 Среднее количество слов с пассивным залогом на предложение
def PassVerb(text):
    number_of_sentences = sent_tokenize(text)
    words = text.split()
    res = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'VERB' or p.tag.POS == 'PRTF' or p.tag.POS == 'PRTS':
            res.append(p.tag.voice)
    S = res.count('pssv')
    patternSC = r'ся\s|сь\s' #эта досчитывалка регуляркой сделана изз-за кривизны pymorphy2
    match_pattern = re.findall(patternSC, text)
    total = S + len(match_pattern)
    return float ( total / len(number_of_sentences) )  

#13 Средняя длина слова в символах
def AvWord(text):
    text = ''.join([i for i in text if not i.isdigit()])
    for c in string.punctuation:
        text = text.replace(c,"")
    words = textstat.lexicon_count(text)  
    text1 = ''.join(e for e in text if e.isalnum())
    letters = len(text1)
    average_word_length = float(letters / words)
    return average_word_length 

#14 Доля слов текста вне списка A1 + A2
def AbsWordAA(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    words = textstat.lexicon_count(test)  
    test = nlp(test)
    lemmatized = []
    for token in test:
        if token.pos_ != 'PUNCT' and token.pos_ != 'NUM':
            lemma = token.lemma_.strip()
            lemmatized.append(lemma)
    S = []
    for word in lemmatized:
        if word != '':
            S.append(word)
    S = [re.sub(u'[Ёё]', u'е', word, flags=re.U|re.I) for word in S]
    Res = [x for x in S if x not in A1]

    return  len(Res) / words

#15 Доля слов текста вне списка лексического минимума Минобрнауки B1
def AbsWordB(text):
    BBB = A1 + B1
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    words = textstat.lexicon_count(test)  
    test = nlp(test)
    lemmatized = []
    for token in test:
        if token.pos_ != 'PUNCT' and token.pos_ != 'NUM':
            lemma = token.lemma_.strip()
            lemmatized.append(lemma)
    S = []
    for word in lemmatized:
        if word != '':
            S.append(word)
    S = [re.sub(u'[Ёё]', u'е', word, flags=re.U|re.I) for word in S]
    Res = [x for x in S if x not in BBB]

    return  len(Res) / words

#16 Доля низкочастотных/редких слов
def LFW(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words1 = textstat.lexicon_count(test)  
    words = test.split()
    res = []
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    new = [re.sub(u'[Ёё]', u'е', word, flags=re.U|re.I) for word in res]
    filtered_tokens = []
    for token in new:
        if token not in stopwords:
            filtered_tokens.append(token)


    final = [x for x in filtered_tokens if x not in full_dict or x in dictionary_freq]

    return len(final) / words1

#17 Type token ratio
def TTR(text):
    test = text.lower()
    ds = DiversityStats(text)

    return ds.ttr

#18 Root Type-Token Ratio
def RootTTR(text):
    test = text.lower()
    ds = DiversityStats(text)

    return ds.rttr

#19 Corrected Type-Token Ratio
def CorrTTR(text):
    test = text.lower()
    ds = DiversityStats(text)

    return ds.cttr

#20 Herdan Type-Token Ratio (logarithmic)
def HTTR(text):
    test = text.lower()
    ds = DiversityStats(text)

    return ds.httr


#21 Summer Type-Token Ratio
def STTR(text):
    test = text.lower()
    ds = DiversityStats(text)

    return ds.sttr

#22 Mean Segmental Type-Token Ratio
def MSTTR(text):
    test = text.lower()
    ds = DiversityStats(text)

    return ds.msttr

#23 Measure of Textual Lexical Diversity
def MTLD(text):
    test = text.lower()
    ds = DiversityStats(text)

    return ds.mtld

#24 Доля слов длиннее 6 символов 
def Lex6(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    words = textstat.lexicon_count(test)  
    test = nlp(test)
    lemmatized = []
    for token in test:
        if token.pos_ != 'PUNCT' and token.pos_ != 'NUM':
            lemma = token.lemma_.strip()
            lemmatized.append(lemma)
    S = []
    for word in lemmatized:
        if word != '':
            S.append(word)
    K = []
    for lemma in S:
        K.append(len(lemma))
    N = []
    for number in K:
        if number > 6:
            N.append(number) 
    return float (len(N) / words)

#25 Доля слов длиннее 8 символов
def Lex8(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    words = textstat.lexicon_count(test)  
    test = nlp(test)
    lemmatized = []
    for token in test:
        if token.pos_ != 'PUNCT' and token.pos_ != 'NUM':
            lemma = token.lemma_.strip()
            lemmatized.append(lemma)
    S = []
    for word in lemmatized:
        if word != '':
            S.append(word)
    K = []
    for lemma in S:
        K.append(len(lemma))
    N = []
    for number in K:
        if number > 8:
            N.append(number) 
    return float (len(N) / words)

#26 Доля слов длиннее 10 символов
def Lex10(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    words = textstat.lexicon_count(test)  
    test = nlp(test)
    lemmatized = []
    for token in test:
        if token.pos_ != 'PUNCT' and token.pos_ != 'NUM':
            lemma = token.lemma_.strip()
            lemmatized.append(lemma)
    S = []
    for word in lemmatized:
        if word != '':
            S.append(word)
    K = []
    for lemma in S:
        K.append(len(lemma))
    N = []
    for number in K:
        if number > 10:
            N.append(number) 
    return float (len(N) / words)

#27 Доля слов длиннее 12 символов
def Lex12(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    words = textstat.lexicon_count(test)  
    test = nlp(test)
    lemmatized = []
    for token in test:
        if token.pos_ != 'PUNCT' and token.pos_ != 'NUM':
            lemma = token.lemma_.strip()
            lemmatized.append(lemma)
    S = []
    for word in lemmatized:
        if word != '':
            S.append(word)
    K = []
    for lemma in S:
        K.append(len(lemma))
    N = []
    for number in K:
        if number > 12:
            N.append(number) 
    return float (len(N) / words)

#28 Доля отглагольных существительных
def VN(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words1 = textstat.lexicon_count(test)  
    words = test.split()
    res = []
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    new = [re.sub(u'[Ёё]', u'е', word, flags=re.U|re.I) for word in res]
    S = []
    for word in new:
        if word.endswith(('тие','ние','ация','аж','ыш','изм','ура','ур','ище','ищ','ство','тва','тель')) or  word in VN1:
            S.append(word)
    return len(S) / words1

#29 Доля существительных в родительном падеже
def GenN(text):
    test = text.lower()
    words = test.split()
    nouns = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'NOUN':
            nouns.append(p.tag.case)
    gents = []
    for case in nouns:
        if case == 'gent' or case == 'gen2':
            gents.append(case)
    if len(nouns) == 0:
        return 0
    else:
        return len(gents)/len(nouns)
    
#30 Средняя длина содержательного слова в морфемах
def WLM(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words1 = textstat.lexicon_count(test)  
    words = test.split()
    res = []
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    new = [re.sub(u'[Ёё]', u'е', word, flags=re.U|re.I) for word in res]
    S = []
    for word in new:
        p = morph.parse(word)[0]
        if p.tag.POS == 'NOUN' or p.tag.POS == 'VERB' or p.tag.POS == 'INFN' or p.tag.POS == 'PRTF' or p.tag.POS == 'PRTS' or p.tag.POS == 'ADVB':
            S.append(word)

    A = []
    M = []
    for word in S:
        for key, value in Tikh_dict.items():
            if key == word:
                A.append(Tikh_dict[key])
                M.append(word)

    return float (sum(A) / len(M))

#31 Доля существительных 
def POSN(text):
    test = text.lower()
    words = test.split()
    words1 = textstat.lexicon_count(test)  
    nouns = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'NOUN':
            nouns.append(word)
    return float (len(nouns) / words1)

#32 Доля глаголов 
def POSV(text):
    test = text.lower()
    words = test.split()
    words1 = textstat.lexicon_count(test)  
    verbs = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'VERB' or p.tag.POS == 'INFN':
            verbs.append(word)
    return float (len(verbs) / words1)

#33 Доля прилагательных 
def POSAdj(text):
    test = text.lower()
    words = test.split()
    words1 = textstat.lexicon_count(test)  
    adjectives = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'ADJF' or p.tag.POS == 'ADJS':
            adjectives.append(word)
    return float (len(adjectives) / words1)

#34 Доля наречий 
def POSA(text):
    test = text.lower()
    words = test.split()
    words1 = textstat.lexicon_count(test)  
    adverbs = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'ADVB':
            adverbs.append(word)
    return float (len(adverbs) / words1)

#35 Доля местоимений 
def POSPr(text):
    test = text.lower()
    words = test.split()
    words1 = textstat.lexicon_count(test)  
    pronouns = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'NPRO':
            pronouns.append(word)
    return float (len(pronouns) / words1) 

#36 Эвфонический индекс по мотивам Индекса Иванова
def EuphInd(text):
    test = ''.join([i for i in text if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    text1 = ''.join(e for e in test if e.isalnum())
    total_letters = len(text1)

    sentences = sent_tokenize(text)

    A = []
    for sentence in sentences:
        for item in NDU_dict:
            if item in sentence:
                A.append(item)

    S = []
    for sentence in sentences:
        for item in NDT_dict:
            if item in sentence:
                S.append(item)

    return float ((len(A) + len(S)) / total_letters)

#37 Доля абстрактных существительных 
def AbsN(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words1 = textstat.lexicon_count(test)  
    words = test.split()
    nouns = []
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'NOUN':
            nouns.append(p.normal_form) 

    new = [re.sub(u'[Ёё]', u'е', word, flags=re.U|re.I) for word in nouns]
    S = []
    for word in new:
        if word.endswith(('ье','ие','ие','ство','честв','ация','ость','изм','изна','ота','щина','ня','ика','тива')):
            S.append(word)

    if len(nouns) == 0:
        return 0
    else:
        return float (len(S)/len(nouns))


#38 Вес «канцелярита»
def Kanz(text):
    test = text.lower()
    dict_kanz = ('в условиях','в части','в связи','в силу','по мере','за счет','в отношении','на основании','вопреки','ввиду','наподобие','в меру','стороны','наряду','в отличие от','внутри','вокруг','впереди','напротив','поверх','посреди','сверху','свыше','сзади','вслед','наперекор','вопреки','согласно','соответственно','между','помимо','благодаря','включая','исключая','спустя','кончая','не считая','начиная с','несмотря','в заключение','в заключении','по поводу','по причине','в продолжение','в продолжении','по случаю','в соответствии','за счет','за счёт','в течение', 'этих условиях','в настоящее время','на текущий момент','имеет место быть', 'имело место быть','имели место быть','имел место быть','имела место быть','следует отметить','принимая во внимание','гарантированно','в установленном порядке')

    sentences = sent_tokenize(text)
    A = []
    for sentence in sentences:
        for item in dict_kanz:
            if item in sentence:
                A.append(item)
    
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words1 = textstat.lexicon_count(test)  
    words = test.split()
    poses = []
    for word in words:
        p = morph.parse(word)[0]
        poses.append(p.tag.POS) 

    mist = []
    i = 0
    while i < len(poses)-1:
        if poses[i] == 'NOUN' and poses[i+1] == 'NOUN':
            mist.append('!')
            i += 1
        else:
            i += 1
    
    if len(poses) == 0:
        return 0
    else:
        return float ( (2*len(mist) + len(A)) / len(poses)) 

#39 Вес меток неопределенности 
def Hedging(text):
    test = text.lower()

    uncert_tags = ('несколько','немного', 'немногим', 'наверное','вероятно','видимости','возможно','степени','сомнительно','сомнения','сомнение','предположительно','якобы','как утверждают','как утверждается','будто бы','как утверждалось','по утверждениям','слегка','чуть-чуть','незначительно','не в полной мере','неуверенно','может','могло','могла','мог','могли','тем не менее','маловероятно','по-видимому','почти','приблизительно','как бы','на всякий случай','тем не менее','кажется','казаться','казался','представлялся','представляется','казалось','казалась','представлялась','представлялось', 'недооцененный', 'недооценен', 'мера', 'мере', 'мерой', 'меру', 'с учетом', 'с учётом', 'учитывая', 'условиях', 'в том числе', 'по мере', 'неметился', 'намечающийся', 'наметившиеся', 'в целом', 'наряду', 'наряду')

    sentences = sent_tokenize(test)
    A = []
    for sentence in sentences:
        for item in uncert_tags:
            if item in sentence:
                A.append(item)

    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words1 = textstat.lexicon_count(test)  

    return float (len(A) / words1)

#40 Вес меток «водности» текста (идеально до 15-20%)
def WTR(text):
    test = text.lower()
    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words1 = textstat.lexicon_count(test)  
    words = test.split()
    res = []
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    new = [re.sub(u'[Ёё]', u'е', word, flags=re.U|re.I) for word in res]

    unfiltered_tokens = []
    for token in new:
        if token  in stopwords:
            unfiltered_tokens.append(token)
    
    return float (len(unfiltered_tokens) / words1)

#41 Вес сущностей
def Entit(text):
    doc2 = nlp2(text) 
    S = doc2.entities
    sentences = sent_tokenize(text)

    return float (len(S) / len(sentences))

#42 Среднее косинусное расстояние между предложениями
def cohLSA(text):
    sentences = sent_tokenize(text)
    i = 0
    S = []
    for sent in sentences:
        while i < len(sentences)-1:
            s1 = nlp(sentences[i])
            s2 = nlp(sentences[i+1])
            s = s1.similarity(s2)
            S.append(s)
            i += 1

    return float (np.mean(S))


#43 Адаптированный ARI для экономических текстов на русском языке по шкале от 1 до 10
def ARI(text):
    text = text.lower()
    
    words = textstat.lexicon_count(text, removepunct=True)
    number_of_sentences = sent_tokenize(text)
    sentences = len(number_of_sentences)
    average_sentence_length = float(words / sentences)

    text = ''.join([i for i in text if not i.isdigit()])
    for c in string.punctuation:
        text = text.replace(c,"")
    words = textstat.lexicon_count(text)  
    text1 = ''.join(e for e in text if e.isalnum())
    letters = len(text1)
    average_word_length = float(letters / words)
    
    result = 19.025 - 2.184*average_word_length - 0.048*average_sentence_length
    
    return result 

#44 Метки дискурса / асемантические языковые единицы
def DiscMar(text):
    test = text.lower()

    disc_dict = ('буквально','ведь','видишь','видите','в некотором роде','в общем','в общем-то','в принципе','вот','знаешь','знаете','итак','и так далее','как бы','как говорится','как сказать','короче','на самом деле','ну','послушай','правда','прямо','скажем','вообще','собственно говоря','так вот','так сказать','типа','только','то есть','я имею в виду','опять','всего лишь','пожалуй','к счастью','к сожалению','кстати','между прочим','кроме того','также','таким образом','но','однако','хотя','в частности','во-первых','во-вторых','прежде всего','в-третьих','соответственно')

    test = ''.join([i for i in test if not i.isdigit()])
    for c in string.punctuation:
        test = test.replace(c,"")
    test = test.replace('–', "")
    test = test.replace('—', "")
    words = test.split()

    R = []  
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'PRCL' or p.tag.POS == 'PRCL':
            R.append(word)

    L = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        for item in disc_dict:
            if item in sentence and item not in S:
                L.append(item)
    J = L + R
 
    return float(len(J) / len(sentences) )

In [4]:
#Загружаем нейросетевую модель

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim=32, num_heads=2, ff_dim=32, rate=0.1, name = None, **kwargs):
        super(TransformerBlock, self).__init__(name=name)
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        super(TransformerBlock, self).__init__(**kwargs)
        
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'num_heads': 2,
            'ff_dim': 32,
            'embed_dim': 32
            })
        return config


    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen=300, vocab_size=40000, embed_dim=32, name = None, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(name=name)
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)


    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'vocab_size': 40000,
            'maxlen': 300,
            'embed_dim': 32
            })
        return config

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


#загружаем словарь 
with open('tokenizer13.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

#загружаем саму модель
model = tf.keras.models.load_model('NN_model_May21.h5', custom_objects= {'TokenAndPositionEmbedding': TokenAndPositionEmbedding, 'TransformerBlock': TransformerBlock})

nb_classes = 10
maxlen = 300



In [5]:
#Загрузка текста для обработки

Testing_text = open("test.txt", 'r')
Testing_text = Testing_text.read()


In [6]:
#Обработка текста

data = {'Text': [Testing_text], 'SenLen': SenLen(Testing_text), 'PTH': PTH(Testing_text),'SubCon': SubCon(Testing_text),'CoordC': CoordC(Testing_text),'NP': NP(Testing_text),'VP': VP(Testing_text),
'NUM': NUM(Testing_text),'DiffSynt': DiffSynt(Testing_text),'LSDaoust': LSDaoust(Testing_text),'PPos': PPos(Testing_text),'PrepR': PrepR(Testing_text),'PassVerb': PassVerb(Testing_text),'AvWord': AvWord(Testing_text),
'AbsWordAA': AbsWordAA(Testing_text),'AbsWordB': AbsWordB(Testing_text),'LFW': LFW(Testing_text),'TTR': TTR(Testing_text),'RootTTR': RootTTR(Testing_text),'CorrTTR': CorrTTR(Testing_text),'HTTR': HTTR(Testing_text),
'STTR': STTR(Testing_text),'MSTTR': MSTTR(Testing_text),'MTLD': MTLD(Testing_text),'Lex6': Lex6(Testing_text),'Lex8': Lex8(Testing_text),'Lex10': Lex10(Testing_text),'Lex12': Lex12(Testing_text),
'VN': VN(Testing_text),'GenN': GenN(Testing_text),'WLM': WLM(Testing_text),'POSN': POSN(Testing_text),'POSV': POSV(Testing_text),'POSAdj': POSAdj(Testing_text),
'POSA': POSA(Testing_text),'POSPr': POSPr(Testing_text),'EuphInd': EuphInd(Testing_text),'AbsN': AbsN(Testing_text),'Kanz': Kanz(Testing_text),'Hedging': Hedging(Testing_text),'WTR': WTR(Testing_text),
'Entit': Entit(Testing_text),'cohLSA': cohLSA(Testing_text),'ARI': ARI(Testing_text),'DiscMar': DiscMar(Testing_text)}

df = pd.DataFrame(data)

#print(df)



In [7]:
#Оценка текста нейросетевой моделью 

test_sequences = tokenizer.texts_to_sequences(df['Text'])
x_val = pad_sequences(test_sequences, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

test_df = df.drop('Text', axis=1)

X_test_scaled = ss.transform(test_df)

X_test_scaled = np.nan_to_num(X_test_scaled)



y_pred = model.predict([x_val, X_test_scaled])

y_pred = np.argmax(y_pred, axis=1)+1

#print(y_pred)

D:\Python\379\sklearn\base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [8]:
#Вывод итоговых результатов оценки текста

print("//Оптимальное значение удобочитаемости - выше 6. Такой текст доступен большей части населения.//")

print()

print("Метрики вашего текста (целевой уровень - 5-6)")

print('ЯСНОСТЬ ФОРМЫ (Классическая оценка удобочитаемости на основе средней длины слов и предложений):', ARI(Testing_text))

x = round(ARI(Testing_text))

if x <= 1:
    print("Ваш текст доступен", "докторам экономических наук")
elif x == 2:
    print("Ваш текст доступен", "магистрам, аспирантам экономических специальностей")
elif x == 3:
    print("Ваш текст доступен", "выпускникам вузов экономических специальностей")  
elif x == 4:
    print("Ваш текст доступен", "выпускникам вузов неэкономических специальностей") 
elif x == 5:
    print("Ваш текст доступен", "учащимся вузов, а также людям со средним специальным образованием")   
elif x == 6:
    print("Ваш текст доступен", "выпускникам школ")   
elif x == 7:
    print("Ваш текст доступен", "учащимся старших классов")
elif x == 8:
    print("Ваш текст доступен", "учащимся старших средних классов")
elif x == 9:
    print("Ваш текст доступен", "учащимся младших средних классов")
elif x > 10:
    print("Ваш текст доступен", "младшим школьникам")         


print('ЯСНОСТЬ СОДЕРЖАНИЯ (классификатор нейронной сети):', int(y_pred))

x = int(y_pred)

if x <= 1:
    print("Ваш текст доступен", "докторам экономических наук")
elif x == 2:
    print("Ваш текст доступен", "магистрам, аспирантам экономических специальностей")
elif x == 3:
    print("Ваш текст доступен", "выпускникам вузов экономических специальностей")  
elif x == 4:
    print("Ваш текст доступен", "выпускникам вузов неэкономических специальностей") 
elif x == 5:
    print("Ваш текст доступен", "учащимся вузов, а также людям со средним специальным образованием")   
elif x == 6:
    print("Ваш текст доступен", "выпускникам школ")   
elif x == 7:
    print("Ваш текст доступен", "учащимся старших классов")
elif x == 8:
    print("Ваш текст доступен", "учащимся старших средних классов")
elif x == 9:
    print("Ваш текст доступен", "учащимся младших средних классов")
elif x > 10:
    print("Ваш текст доступен", "младшим школьникам")     

print()

print("///Отдельные характеристики текста, улучшение которых позволит повысить удобочитаемость///")

print("Из них топ-7 важнейших:")
print("1. Средняя длина слова в символах:", df.AvWord[0], "//Оптимально (среднее значение для 6го уровня) < 5.98365741523809")
print("2. Средняя длина предложения в словах:", df.SenLen[0], "//Оптимально < 16.02344739873717")
print("3. Доля существительных:", df.POSN[0], "//Оптимально < 0.25078608578531747")
print("4. Доля слов длиннее 8 символов:", df.Lex8[0], "//Оптимально < 0.22357996388384185")
print("5. Доля отглагольных существительных:", df.VN[0], "//Оптимально < 0.0716975295562396")
print("6. Среднее количество слов с пассивным залогом или возвратной формой на предложение:", df.PassVerb[0], "//Оптимально < 0.5294425959430092")
print("7. Среднее число предложений, отягощенных причастиями и деепричастиями:", df.DiffSynt[0], "//Оптимально < 0.32258362278746855")


print()

print("Прочие:")
print("Средняя длина дерева синтаксического разбора на предложение:", df.PTH[0], "//Оптимально < 5.392461876298687")
print("Среднее число подчинительных союзов на предложение:", df.SubCon[0], "//Оптимально < 0.24590303381930237")
print("Среднее число сочинительных союзов на предложение:", df.CoordC[0], "//Оптимально < 0.22826876067946722")
print("Доля именных деревьев в среднем на предложение:", df.NP[0], "//Оптимально < 0.07225599758247445")
print("Доля глагольных деревьев в среднем на предложение:", df.VP[0], "//Оптимально > 0.7942089845265379")
print("Среднее число цифр на предложение:", df.NUM[0], "//Оптимально < 0.29722749684907984")
print("Доля предложений, длина которых превышает 25 слов:", df.LSDaoust[0], "//Оптимально < 0.1459197808822559")
print("Среднее число слов до подлежащего или сказуемого на предложение:", df.PPos[0], "//Оптимально < 5.181730529685941")
print("Доля «:», «(», «)» и «;» в знаках препинания:", df.PrepR[0], "//Оптимально < 0.031730535428459515")
print("Доля слов текста вне списка лексического минимума Минобрнауки A2 (базовый):", df.AbsWordAA[0], "//Оптимально < 0.5219993459811691")
print("Доля слов текста вне списка лексического минимума Минобрнауки B1 (первый сертификационный уровень):", df.AbsWordB[0], "//Оптимально < 0.43205610636445063")
print("Доля низкочастотных слов (условно терминов):", df.LFW[0], "//Оптимально < 0.10336615931506997")
print("Лексическое разнообразие 1:", df.TTR[0], "//Оптимально > 0.7540145922582578")
print("Лексическое разнообразие 2:", df.RootTTR[0], "//Оптимально > 11.55135020226921")
print("Лексическое разнообразие 3:", df.CorrTTR[0], "//Оптимально > 8.168038059885133")
print("Лексическое разнообразие по формуле Herdan:", df.HTTR[0], "//Оптимально > 0.9479673799357601")
print("Лексическое разнообразие по формуле Somers:", df.STTR[0], "//Оптимально > 0.9377959894865681")
print("Модификация метрики TTR с использованием сегментирования:", df.MSTTR[0], "//Оптимально > 0.8986683355803643")
print("Лексическое разнообразие по формуле McCarthy and Jarvis:", df.MTLD[0], "//Оптимально > 262.3759935124898")
print("Доля слов длиннее 6 символов:", df.Lex6[0], "//Оптимально < 0.4110677383813233")
print("Доля слов длиннее 10 символов:", df.Lex10[0], "//Оптимально < 0.10218454534254123")
print("Доля слов длиннее 12 символов:", df.Lex12[0], "//Оптимально < 0.03922159561975452")
print("Нагруженность канцеляритом:", df.Kanz[0], "//Оптимально < 0.22059938451417027")
print("Доля существительных в родительном падеже:", df.GenN[0], "//Оптимально < 0.3535513977365723")
print("Средняя длина содержательного слова  в морфемах:", df.WLM[0], "//Оптимально < 1.971563864653573")
print("Доля глаголов:", df.POSV[0], "//Оптимально > 0.10463763007050884")
print("Доля прилагательных:", df.POSAdj[0], "//Оптимально ~ 0.1212542934468348")
print("Доля наречий:", df.POSA[0], "//Оптимально ~ 0.038119436988495584")
print("Доля местоимений:", df.POSPr[0], "//Оптимально ~ 0.033903158016570595")
print("Эвфонический индекс на основе Иванов (2013):", df.EuphInd[0], "//Оптимально > 0.16430284019519137")
print("Доля абстрактных существительных:", df.AbsN[0], "//Оптимально < 0.18424762356036078")
print("Вес меток неопределенности:", df.Hedging[0], "//Оптимально < 0.011028098947534001")
print("Вес меток «водности» текста (стоп-слова, не несущие содержательного смысла):", df.WTR[0], "//Оптимально ~ 0.4213621630589011")
print("Плотность сущностей (entities), или устойчивых понятий в тексте:", df.Entit[0], "//Оптимально < 1.0498635118245612")
print("Связность предложений внутри текста:", df.cohLSA[0], "//Оптимально > 0.5291031318188171")
print("Метки разговорного стиля:", df.DiscMar[0], "//Оптимально > 1.3044459743870107")

print()

print("Подробная информация о моделях и характеристиках текста представлена в Evstigneeva, A. and Sidorovskiy, M. (2021). Assessment of Clarity of Bank of Russia Monetary Policy Communication by Neural Network Approach. Russian Journal of Money and Finance, 80(3), pp. 3–33.",
"doi: 10.31477/rjmf.202103.03")


//Оптимальное значение удобочитаемости - выше 6. Такой текст доступен большей части населения.//

Метрики вашего текста:
4.60862284014189 - Классическая оценка удобочитаемости на основе средней длины слов и предложений
Ваш текст доступен учащимся вузов, а также людям со средним специальным образованием
2 - Удобочитаемость, оценка нейросетевой модели
Ваш текст доступен магистрам, аспирантам экономических специальностей

///Отдельные характеристики текста, улучшение которых позволит повысить удобочитаемость///
Из них топ-7 важнейших:
1. Средняя длина слова в символах: 6.200823892893924 //Оптимально (среднее значение для 6го уровня) < 5.98365741523809
2. Средняя длина предложения в словах: 18.203703703703702 //Оптимально < 16.02344739873717
3. Доля существительных: 0.2517802644964395 //Оптимально < 0.25078608578531747
4. Доля слов длиннее 8 символов: 0.25798146240988673 //Оптимально < 0.22357996388384185
5. Доля отглагольных существительных: 0.08770108977685522 //Оптимально < 0.0716975295